In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD

In [ ]:

def VGG16Model():
  model = keras.Sequential()
  model.add(keras.Input(shape=(32,32,3)))  # 32*32 RGB images
  model.add(layers.Conv2D(64, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(64, 3, strides=1, padding="same",activation="relu"))
  model.add(layers.MaxPooling2D(2))

  model.add(layers.Conv2D(128, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(128, 3, strides=1, padding="same",activation="relu"))
  model.add(layers.MaxPooling2D(2))

  model.add(layers.Conv2D(256, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(256, 3, strides=1, padding="same",activation="relu"))
  model.add(layers.MaxPooling2D(2))

  model.add(layers.Conv2D(512, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(512, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(512, 3, strides=1, padding="same",activation="relu"))
  model.add(layers.MaxPooling2D(2))

  model.add(layers.Conv2D(512, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(512, 3, strides=1, padding= "same", activation="relu"))
  model.add(layers.Conv2D(512, 3, strides=1, padding="same",activation="relu"))
  model.add(layers.MaxPooling2D(2))

  model.add(layers.Flatten())
  model.add(layers.Dense(4096,activation='relu'))
  model.add(layers.Dense(4096,activation='relu'))
  model.add(layers.Dense(10,activation='softmax'))

  # compile model
  opt = SGD(learning_rate=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	
# Can you guess what th#e current output shape is at this point? Probably not.
# Let's just print it:
  model.summary()

  return model

In [ ]:
##load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [ ]:
import numpy as np
import pickle
def load_experiment13_dataset():
	# load dataset
	fo = open('drive/MyDrive/data-transformed.pkl', 'rb')
	h_data = pickle.load(fo)
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	h_imgs = h_data['transformed-images'][10000:40000]
	h_labels = h_data['labels'][10000:40000]
	trainX_new = np.concatenate((trainX[:30000],h_imgs))
	h_labels = to_categorical(h_labels)
	trainY_new = np.concatenate((trainY[:30000],h_labels))
	return trainX_new, trainY_new, testX[:10000], testY[:10000]

In [ ]:
# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + 'exp13_plot.png')
	pyplot.savefig('drive/MyDrive/Experiments/Experiment-13.png')
	pyplot.close()

In [ ]:
# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_experiment13_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = VGG16Model()
	# fit model
	history = model.fit(trainX, trainY, epochs=100, batch_size=64, validation_data=(testX, testY), verbose=0)
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
	return history
# entry point, run the test harness

m = run_test_harness()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 conv2d_37 (Conv2D)          (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 conv2d_39 (Conv2D)          (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 8, 8, 128)        0         
 g2D)                                                 